## Installation and Imports

In [2]:
!pip install -r ../requirements.txt

^C


In [2]:
!pip install matplotlib

  Using cached matplotlib-3.10.1-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached pillow-11.1.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.1-cp311-cp311-win_amd64.whl (8.1 MB)
Using cached contourpy-1.3.1-cp311-cp311-win_amd64.whl (219 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 24.9 MB/s eta 0:00:00
Using cached kiwisolver-1.4.8-cp311-cp311-win_amd64.whl (71 kB)
Using cached pillow-11.1.0-cp311-cp311-win_amd64.whl (2.6 MB)
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)


In [4]:
!python -m pip install ../rlang-0.2.5-py3-none-any.whl.

Processing c:\users\zakir\documents\github\declarer\rlang-0.2.5-py3-none-any.whl
INFO: pip is looking at multiple versions of rlang to determine which version is compatible with other requirements. This could take a while.


ERROR: Could not find a version that satisfies the requirement ale-py==0.7.3 (from rlang) (from versions: 0.8.0, 0.8.1, 0.9.0, 0.9.1, 0.10.0, 0.10.1, 0.10.2)
ERROR: No matching distribution found for ale-py==0.7.3


In [5]:
!pip install gymnasium
!pip install "gymnasium[toy-text]"

  Using cached gymnasium-1.1.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
Using cached gymnasium-1.1.1-py3-none-any.whl (965 kB)
Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl (12.9 MB)
  Using cached pygame-2.6.1-cp311-cp311-win_amd64.whl.metadata (13 kB)
Using cached pygame-2.6.1-cp311-cp311-win_amd64.whl (10.6 MB)


In [3]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

In [3]:
env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=False, render_mode="human")

In [4]:
observation, info = env.reset(seed=0)

In [8]:
env.step(2)

(1, 0.0, False, False, {'prob': 1.0})

In [6]:
env.render()

In [16]:
env.close()

In [11]:
env.unwrapped.desc

array([[b'S', b'F', b'F', b'F'],
       [b'F', b'H', b'F', b'H'],
       [b'F', b'F', b'F', b'H'],
       [b'H', b'F', b'F', b'G']], dtype='|S1')

In [13]:
def get_tile_info(env):
    desc = env.unwrapped.desc
    tiles_info = {
        'Start': [],
        'Frozen': [],
        'Hole': [],
        'Goal': []
    }

    for row in range(desc.shape[0]):
        for col in range(desc.shape[1]):
            tile = desc[row, col].decode('utf-8')
            if tile == 'S':
                tiles_info['Start'].append([row, col])
            elif tile == 'F':
                tiles_info['Frozen'].append([row, col])
            elif tile == 'H':
                tiles_info['Hole'].append([row, col])
            elif tile == 'G':
                tiles_info['Goal'].append([row, col])

    return tiles_info

In [12]:
tiles_info = get_tile_info(env)

print("Tiles Information:")
print("Start Tiles:", tiles_info['Start'])
print("Frozen Tiles:", tiles_info['Frozen'])
print("Hole Tiles:", tiles_info['Hole'])
print("Goal Tiles:", tiles_info['Goal'])

Tiles Information:
Start Tiles: [[0, 0]]
Frozen Tiles: [[0, 1], [0, 2], [0, 3], [1, 0], [1, 2], [2, 0], [2, 1], [2, 2], [3, 1], [3, 2]]
Hole Tiles: [[1, 1], [1, 3], [2, 3], [3, 0]]
Goal Tiles: [[3, 3]]


In [14]:
def write_to_rlang_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)

filename = "frozen_lake.rlang"
content = f"Tiles Information:\nStart Tiles: {tiles_info['Start']}\nFrozen Tiles: {tiles_info['Frozen']}\nHole Tiles: {tiles_info['Hole']}\nGoal Tiles: {tiles_info['Goal']}"
write_to_rlang_file(filename, content)

NameError: name 'tiles_info' is not defined

In [19]:
def env_to_rlang(env):
    tiles_info = get_tile_info(env)
    desc = env.unwrapped.desc
    rows, cols = desc.shape
    with open("frozen_lake.rlang", "w") as file:
        file.write(f"import vocab.json\n")
        file.write(f"Constant frozen_locs := {tiles_info['Frozen']}\n")
        file.write(f"Constant hole_locs := {tiles_info['Hole']}\n")
        file.write(f"Factor position := S[0,1]\n")
        file.write(f"Factor row := position[0]\n")
        file.write(f"Factor col := position[1]\n")
        file.write(f"Factor state := row * {cols} + col\n")
        file.write(f"Proposition reached_goal := state == {tiles_info['Goal'][0][0] * cols + tiles_info['Goal'][0][1]}\n")
        file.write(f"Proposition in_hole := position in hole_locs\n")
        file.write(f"Action left := 0\n")
        file.write(f"Action down := 1\n")
        file.write(f"Action right := 2\n")
        file.write(f"Action up := 3\n")
        file.write(f"Effect movement_effect:\n")
        file.write(f"   if A == left and col > 0:\n")
        file.write(f"     position += [-1,0]\n")
        file.write(f"   elif A == down and row < {rows - 1}:\n")
        file.write(f"     position += [0,1]\n")
        file.write(f"   elif A == right and col < {cols - 1}:\n")
        file.write(f"     position += [1,0]\n")
        file.write(f"   elif A == up and row > 0:\n")
        file.write(f"     position += [0,-1]\n")
        file.write(f"Effect reward effect:\n")
        file.write(f"  if reached_goal then\n")
        file.write(f"    Reward 1\n")
        file.write(f"  else if in_hole then\n")
        file.write(f"    Reward 0\n")
        file.write(f"  else\n")
        file.write(f"    Reward 0\n")
        file.write(f"Effect main:\n")
        file.write(f"  -> movement_effect\n")
        file.write(f"  -> reward effect\n")
        file.write(f"Policy lake_policy:\n")
        file.write(f"  if not reached_goal:\n")
        file.write(f"    if row < goal_y:\n")
        file.write(f"      Execute down\n")
        file.write(f"    elif col < goal_x:\n")
        file.write(f"      Execute right\n")        

In [20]:
env_to_rlang(env)

In [2]:
!pip install tensorflow

^C


  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached wrapt-1.17.2-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-win_amd64.whl.metadata (22 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl.metadata (14 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Usin

QL